# Liste des problemes
- Impossible d'importer le module standard "LZMA"

In [1]:
using Pkg
Pkg.activate("venv")
#Pkg.add("SeaPearl")
#Pkg.add("XML")
Pkg.status()

  Activating project at `~/Documents/3A/Stage 3A/Project/XCSP3-Julia-Parser/venv`


     Project venv v0.1.0
      Status `~/Documents/3A/Stage 3A/Project/XCSP3-Julia-Parser/venv/Project.toml`
  [9c8b4983] LightXML v0.9.0
  [c13076dc] SeaPearl v0.4.3
  [72c71f33] XML v0.2.0


# 1. Convert file

In [2]:
include("src/utils/lzma_utils.jl")

file_name = "Haystacks-ext-05_c18.xml"

lzma_root_dir = "instancesXCSP22/lzma/MiniCSP/"
lzma_file_path = lzma_root_dir * file_name * ".lzma"
xml_root_dir = "instancesXCSP22/xml/MiniCSP/"

convert_lzma_file_to_xml(lzma_file_path, xml_root_dir)

LoadError: LoadError: ArgumentError: Package LZMA not found in current path:
- Run `import Pkg; Pkg.add("LZMA")` to install the LZMA package.

in expression starting at /Users/axelnavarro/Documents/3A/Stage 3A/Project/XCSP3-Julia-Parser/src/utils/lzma_utils.jl:2

# 2. Parsing

In [3]:
using XML

include("src/utils/xml_utils.jl")

file_name2 = "CostasArray-15_mc22.xml"
file_name3 = "NumberPartitioning-20_mc22.xml"
file_name4 = "Haystacks-ext-05_c18.xml"
file_name5 = "Hadamard-17_mc22.xml"
file_name6 = "Eternity-06-06_mc22.xml"
file_name7 = "Crypto1-0-64-0-0x3eb894e67055f622-19_c22.xml"
file_name8 = "Crossword-m18-ogd2008-vg-04-05_c18.xml"
file_name9 = "AircraftLanding-table-airland01_mc22.xml"

file_name = file_name3

xml_root_dir = "instancesXCSP22/xml/MiniCSP/"

file_path = xml_root_dir * file_name

file = open(file_path)
xml_string = read(file, String)
close(file)

# Parser le contenu du fichier XML
doc = parse(xml_string)

# Récupérer le noeud instance 
instance = find_element(doc, "instance")

#Récupérer les noeuds variables et constraints
variables = find_element(instance, "variables")
constraints = find_element(instance, "constraints")

Node Element <constraints> (5 children)

## 2.1. Variables parsing

In [4]:
include("src/parsing/variables.jl")

sort_intervals (generic function with 1 method)

In [5]:
using SeaPearl

abstract type AbstractSeaPearlIntVar{T} end

struct VectorSeaPearlIntVar{T} <: AbstractSeaPearlIntVar{T}
    data::Vector{T}
end

struct MatrixSeaPearlIntVar{T} <: AbstractSeaPearlIntVar{T}
    data::Matrix{T}
end


In [6]:
#Create model
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
dict_var = Dict{String, Any}()

Dict{String, Any}()

In [7]:
for var in children(variables)
    id = attributes(var)["id"]
    if var.tag == "array"
        dict_var[id] = parse_array_variable(var, model, trailer)
    end
end

In [8]:
dict_var

Dict{String, Any} with 3 entries:
  "aux_gb" => SeaPearl.IntVar[aux_gb[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 …
  "x"      => SeaPearl.IntVar[x[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17…
  "y"      => SeaPearl.IntVar[y[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17…

## 2.2. Constraints parsing

In [9]:
include("src/parsing/constraints.jl")

get_constraint_variables (generic function with 1 method)

In [10]:
for con in children(constraints)
    parse_constraint(con, dict_var, model, trailer)
    break
end
println(model.constraints)

SeaPearl.Constraint[SeaPearl.AllDifferent: x[0] != x[1] != x[2] != x[3] != x[4] != x[5] != x[6] != x[7] != x[8] != x[9] != y[0] != y[1] != y[2] != y[3] != y[4] != y[5] != y[6] != y[7] != y[8] != y[9]]


In [44]:
using SeaPearl

struct IntensionNode
    operator::String
    children::Vector{Any}
end

function replace_placeholders(string::String, args)
    for (i, arg) in enumerate(args)
        placeholder = "%$(i-1)"
        string = replace(string, placeholder, string(arg))
    end
    return string
end

const relational_operators = Dict(
    "lt" => SeaPearl.Less,
    "le" => SeaPearl.LessOrEqual,
    "ge" => SeaPearl.GreaterOrEqual,
    "gt" => SeaPearl.Greater,
    "ne" => SeaPearl.NotEqual,
    "eq" => SeaPearl.Equal,
)

const arithmetic_operators = Dict(
    "add" => SeaPearl.Addition,
    "sub" => SeaPearl.Subtraction,
    "mul" => SeaPearl.Multiplication,
    "div" => SeaPearl.Division,
    "mod" => SeaPearl.Modulo,
    "dist" => SeaPearl.Distance,
)


UndefVarError: UndefVarError: Less not defined

In [40]:
function parse_intension_expression(str_constraint, variables, model, trailer)
    # Split the expression into operator and operands
    spl = split(str_constraint, "(", limit=2)
    operator = spl[1]

    rel_bool = haskey(relational_operators, operator)
    ari_bool = haskey(arithmetic_operators, operator)
    if !(rel_bool || ari_bool)
        # If the expression does not have any further parentheses, return it as a variable
        var = get_constraint_variables(str_constraint, variables)
        return var
    end
    operands_str = string(spl[2])[1:end-1]
    # Recursively parse the operands
    operands = []
    start = 1
    balance = 0
    for i = 1:length(operands_str)
        char = operands_str[i]
        if char == '('
            balance += 1
        elseif char == ')'
            balance -= 1
        elseif char == ',' && balance == 0
            var1 = parse_intension_expression(operands_str[start:i-1], variables, model, trailer)
            start = i + 1
            break
        end
    end
    var2 = parse_intension_expression(operands_str[start:end], variables, model, trailer)

    if rel_bool
        model.addConstraint()
        return nothing
    else
        return
    end
end


parse_intension_expression (generic function with 2 methods)

In [45]:
str_constraint = "eq(mul(x[0],x[0]),x[1])"
intension_node = parse_intension_expression(str_constraint, dict_var)


IntensionNode("eq", Any[IntensionNode("mul", Any[SeaPearl.IntVar[x[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20]], SeaPearl.IntVar[x[0] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20]]]), SeaPearl.IntVar[x[1] = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20]]])